In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PtOccupancyFunctions import *

In [ ]:
# def test_getPtOccupancies():
path = "/home/helge/Programs/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/bim_30minTransit_wDiag/180/ITERS/it.0/0.occupancyAnalysis.txt"
path2 = "/home/helge/Programs/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/bim_5minTransit_wDiag/180/ITERS/it.0/0.occupancyAnalysis.txt"
av_occ_30min = getPtOccupancies(path, 540)
t_av_occ_30min = getAverageTimeSeries(av_occ_30min)
av_occ_5min = getPtOccupancies(path2, 540)
t_av_occ_5min = getAverageTimeSeries(av_occ_5min)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)

av_occ_30min.plot(ax=ax1)
ax1.axhline(t_av_occ_30min, ls='--', c='r')
av_occ_5min.plot(ax=ax2)
ax2.axhline(t_av_occ_5min, ls='--', c='r')
ax2.set_xlabel('time (s)')
ax2.set_ylabel('Pt Occupancy')
ax1.set_ylabel('Pt Occupancy')
# plt.savefig('Pt_Occupancy_180vehicles', dpi=200)

plt.show()

In [ ]:
display(av_occ_30min)